# Quantum Portfolio Optimization: QUBO Formulation

This notebook demonstrates how to formulate portfolio optimization problems as Quadratic Unconstrained Binary Optimization (QUBO) problems for quantum annealing devices.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
import os
import time

# Add parent directory to path
sys.path.append(os.path.abspath('..'))

from algorithms.qubo import QuboFormulation
from benchmarks.test_cases import TestCases
from utilities.matrix_conversion import MatrixConversion
from utilities.visualization import PortfolioVisualization

## 1. Introduction to QUBO

Quadratic Unconstrained Binary Optimization (QUBO) is a mathematical form well-suited for quantum annealing devices like D-Wave. The general form is:

\begin{align}
\min_x x^T Q x
\end{align}

where:
- $x \in \{0, 1\}^M$ is a binary decision vector
- $Q$ is an $M \times M$ real-valued matrix

## 2. Loading Test Data

In [ ]:
# Load balanced portfolio test case
portfolio = TestCases.balanced_portfolio()
covariance_matrix = portfolio['covariance_matrix'].values
expected_returns = portfolio['expected_returns'].values
asset_names = portfolio['asset_names']

print(f"Test case: {portfolio['name']}")
print(f"Assets: {asset_names}")
print(f"Expected returns: {expected_returns}")
print("\nCovariance matrix:")
print(covariance_matrix)

## 3. Binary Encoding of Portfolio Weights

To represent continuous portfolio weights in binary form, we use binary expansion:

\begin{align}
w_i = \sum_{j=0}^{b-1} 2^j x_{i,j}
\end{align}

where $b$ is the number of bits used to encode each weight, and $x_{i,j}$ is the $j$-th bit in the binary representation of $w_i$.

In [ ]:
# Set target return
target_return = np.mean(expected_returns)
print(f"Target return: {target_return:.4f}")

# Create QUBO formulation
qubo = QuboFormulation(covariance_matrix, expected_returns, target_return)

# Choose number of bits for encoding
n_bits = 3
print(f"Using {n_bits} bits for encoding each weight")

# Create encoding matrix
encoding_matrix = qubo.encode_weights(n_bits)
print(f"\nEncoding matrix shape: {encoding_matrix.shape}")
print("First few rows:")
print(encoding_matrix[:2, :6])

## 4. Building the QUBO Matrix

The QUBO matrix $Q$ encodes the objective function and constraints:

\begin{align}
f(x) = x^T Q x = \text{risk} + A \cdot \text{penalty}_{\text{return}} + B \cdot \text{penalty}_{\text{budget}} + C \cdot \text{penalty}_{\text{cardinality}}
\end{align}

where $A$, $B$, and $C$ are penalty coefficients.

In [ ]:
# Set penalty coefficients
penalty_coefficients = {
    'return': 10.0,  # Penalty for return constraint
    'budget': 10.0,  # Penalty for budget constraint
    'cardinality': 5.0  # Penalty for cardinality constraint
}

# Build QUBO matrix without cardinality constraint
qubo_matrix = qubo.build_qubo_matrix(encoding_matrix)
print(f"QUBO matrix shape: {qubo_matrix.shape}")

# Visualize QUBO matrix
plt.figure(figsize=(10, 8))
plt.imshow(qubo_matrix, cmap='viridis')
plt.colorbar(label='Coefficient Value')
plt.title('QUBO Matrix Visualization')
plt.xlabel('Binary Variable Index')
plt.ylabel('Binary Variable Index')
plt.tight_layout()
plt.show()

## 5. Incorporating Cardinality Constraints

Cardinality constraints limit the number of assets in the portfolio to $K < N$:

\begin{align}
\sum_{i=1}^{N} \delta_i \leq K
\end{align}

where $\delta_i \in \{0, 1\}$ indicates whether asset $i$ is included in the portfolio.

In [ ]:
# Set cardinality constraint
max_assets = 3
print(f"Maximum number of assets: {max_assets}")

# Build QUBO matrix with cardinality constraint
qubo_matrix_card = qubo.build_qubo_matrix(encoding_matrix, cardinality_constraint=max_assets)

# Visualize difference between QUBO matrices
diff_matrix = qubo_matrix_card - qubo_matrix
print(f"Difference matrix shape: {diff_matrix.shape}")
print(f"Max absolute difference: {np.max(np.abs(diff_matrix)):.4f}")

plt.figure(figsize=(10, 8))
plt.imshow(diff_matrix, cmap='coolwarm')
plt.colorbar(label='Coefficient Difference')
plt.title('Cardinality Constraint Contribution to QUBO Matrix')
plt.xlabel('Binary Variable Index')
plt.ylabel('Binary Variable Index')
plt.tight_layout()
plt.show()

## 6. From QUBO to Ising Model

For some quantum devices, it's useful to convert the QUBO to an Ising model:

\begin{align}
E(s) = \sum_{i,j} J_{ij} s_i s_j + \sum_i h_i s_i + c
\end{align}

where $s_i \in \{-1, 1\}$ are spin variables, $J_{ij}$ are coupling strengths, $h_i$ are local fields, and $c$ is a constant.

In [ ]:
# Convert QUBO to Ising
J, h, offset = MatrixConversion.convert_to_ising(qubo_matrix)

print(f"J matrix shape: {J.shape}")
print(f"h vector shape: {h.shape}")
print(f"Offset: {offset:.4f}")

# Visualize J matrix
plt.figure(figsize=(10, 8))
plt.imshow(J, cmap='coolwarm')
plt.colorbar(label='Coupling Strength')
plt.title('Ising Coupling Matrix (J)')
plt.xlabel('Spin Index')
plt.ylabel('Spin Index')
plt.tight_layout()
plt.show()

# Visualize h vector
plt.figure(figsize=(12, 4))
plt.bar(range(len(h)), h)
plt.title('Ising Local Fields (h)')
plt.xlabel('Spin Index')
plt.ylabel('Field Strength')
plt.tight_layout()
plt.show()

## 7. Solution Extraction and Interpretation

After solving the QUBO problem on a quantum device, we need to extract and interpret the portfolio weights.

In [ ]:
# Simulate binary solution (this would come from a quantum device in practice)
# Generate random binary solution for demonstration
np.random.seed(42)
binary_solution = np.random.randint(0, 2, size=qubo.n_variables)
print(f"Binary solution: {binary_solution}")

# Extract portfolio weights
weights = qubo.extract_portfolio_weights(binary_solution)
print(f"Extracted weights (before normalization): {weights}")

# Normalize weights to sum to 1
normalized_weights = weights / np.sum(weights) if np.sum(weights) > 0 else weights
print(f"Normalized weights: {normalized_weights}")

# Calculate portfolio metrics
result = qubo.evaluate_portfolio(normalized_weights)
print(f"\nPortfolio return: {result['expected_return']:.4f}")
print(f"Portfolio risk: {result['risk']:.4f}")
print(f"Sharpe ratio: {result['sharpe_ratio']:.4f}")
print(f"Number of assets: {result['num_assets']}")

# Plot portfolio weights
plt.figure(figsize=(10, 6))
plt.bar(asset_names, normalized_weights)
plt.title('Portfolio Weights from QUBO Solution')
plt.xlabel('Assets')
plt.ylabel('Weight')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 8. Effect of Precision (Number of Bits)

Let's explore how the number of bits for encoding affects the solution space and precision.

In [ ]:
# Test different bit precisions
bit_precisions = [1, 2, 3, 4, 5]
qubo_sizes = []
weight_precisions = []

for n_bits in bit_precisions:
    # Create encoding matrix
    encoding_matrix = qubo.encode_weights(n_bits)
    
    # Build QUBO matrix
    qubo_matrix = qubo.build_qubo_matrix(encoding_matrix)
    
    # Record size
    qubo_sizes.append(qubo_matrix.shape[0])
    
    # Calculate weight precision
    min_weight = 1.0 / (2**n_bits)
    weight_precisions.append(min_weight)

# Plot results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

ax1.plot(bit_precisions, qubo_sizes, 'o-')
ax1.set_title('QUBO Size vs. Bit Precision')
ax1.set_xlabel('Number of Bits')
ax1.set_ylabel('Number of Binary Variables')
ax1.grid(True, alpha=0.3)

ax2.plot(bit_precisions, weight_precisions, 'o-')
ax2.set_title('Weight Precision vs. Bit Precision')
ax2.set_xlabel('Number of Bits')
ax2.set_ylabel('Minimum Weight Resolution')
ax2.set_yscale('log')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Bit Precision vs. QUBO Size and Weight Resolution:")
for i, n_bits in enumerate(bit_precisions):
    print(f"{n_bits} bits: {qubo_sizes[i]} binary variables, minimum weight = {weight_precisions[i]:.6f}")

## 9. Converting to Quantum Circuits

For gate-based quantum computers, the QUBO or Ising formulation needs to be mapped to a quantum circuit. Here, we'll demonstrate a simplified conversion to a polynomial form suitable for quantum circuit implementation.

In [ ]:
# Convert to binary polynomial
binary_poly = MatrixConversion.convert_to_binary_polynomial(
    covariance_matrix,
    expected_returns,
    target_return,
    n_bits=3
)

print(f"Total number of binary variables: {binary_poly['n_variables']}")
print(f"Number of linear terms: {len(binary_poly['linear_terms'])}")
print(f"Number of quadratic terms: {len(binary_poly['quadratic_terms'])}")
print(f"Constant term: {binary_poly['constant']:.4f}")

# Print a few linear terms
print("\nSample linear terms:")
for i, (var, coef) in enumerate(list(binary_poly['linear_terms'].items())[:5]):
    print(f"x[{var}]: {coef:.4f}")

# Print a few quadratic terms
print("\nSample quadratic terms:")
for i, (vars_str, coef) in enumerate(list(binary_poly['quadratic_terms'].items())[:5]):
    print(f"x[{vars_str}]: {coef:.4f}")

## 10. Conclusion

In this notebook, we've demonstrated how to formulate portfolio optimization problems as Quadratic Unconstrained Binary Optimization (QUBO) problems, which can be solved on quantum annealing devices. We've covered:

1. Binary encoding of portfolio weights
2. Building the QUBO matrix
3. Incorporating constraints via penalty terms
4. Converting to Ising model
5. Extracting and interpreting solutions
6. Effect of bit precision on solution space
7. Converting to binary polynomial form for gate-based quantum computers

In the next notebook, we'll implement the Variational Quantum Eigensolver (VQE) approach for gate-based quantum computers.